In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch


print(torch.cuda.is_available())
if torch.cuda.is_available():
    torch.cuda.empty_cache()
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise fallback to CPU
model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)


pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map=device,
)


True


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [2]:
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
    max_length=200,
    do_sample=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/jq2uw/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:612: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Result: Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.
Daniel: Hello, Girafatron!
Girafatron: Hello, Daniel!
Daniel: What are you doing today?
Girafatron: Today, I am going to go out and find a new friend.
Daniel: That's a great idea! Who do you think you should go find?
Girafatron: I think I should go find a giraffe.
Daniel: That's a good choice! Do you think you can find a giraffe?
Girafatron: Yes, I can! I'm a super-hero!
Daniel: You are a super-hero! I'm sure you can find a giraffe.
Girafatron: Thank


In [3]:
file_id = 1
answer_df_path = "../../bloom_data/answer_df_part"+str(file_id)+".csv"

In [4]:
import pandas as pd
import numpy as np
answer_df = pd.read_csv(answer_df_path)
query_df = pd.read_csv("../../data/fea_df.csv")

if not 'answer_string' in answer_df.columns:
    answer_df['answer_string'] = np.nan
    
print(answer_df.shape)
print(answer_df.iloc[0:10,])

(9647, 3)
     sm_id                                         text_w_eos  answer_string
0  10002yt  20 pound down after 22 week s and 17 week s ou...            NaN
1  1003i3b  tw ana body dysmorphia describing body potenti...            NaN
2  1003xw2  hello fellow brawearing friends. this is going...            NaN
3  10042jf  according to cronometer 100 gram of canola oil...            NaN
4  1004j21  its been two week s since i started strength t...            NaN
5  1008hih  i made panfried chicken thighs ovenroasted pot...            NaN
6  100ash9  i have aversive arfid so i have a very bad fea...            NaN
7  100aza8  original post here httpswww. reddit. comrcicoc...            NaN
8  100bwe6  for context i am early 30s attending a corpora...            NaN
9  100c6y2  i got this from a thrift store and was pleasan...            NaN


In [5]:
system_message = """
You are an AI assistant designed to answer questions.
Please restrict your answer to the exact question and use the exact answer format asked.
Answer should not have implied information. If the answer is yes, always provide related phrases. 
"""

def format_user_message(text):
    question_content = "Does the paragraph mention any of the following topics:\n"
    for i in range(len(query_df)):
        question_content += f"  ({i+1}) {query_df.fea[i]}: {query_df.description[i]}.\n"
    answer_content = "Return answer in format:\n"
    for i in range(len(query_df)): 
        answer_content += f"  ({i+1}) {query_df.fea[i]}: [yes/no], related phrases if any: \n"
    paragragh_content = f"Paragraph: '{text}' \n"
    user_message = question_content + answer_content + paragragh_content
    #print(user_message)
    
    return user_message




In [9]:
# sequences = pipeline(
#    "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
#     max_length=200,
#     do_sample=False,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
# )
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")



batch_size = 1

# Filter for rows where 'answer_string' is NaN
unanswered_df = answer_df[answer_df['answer_string'].isna()]

# Get the indices of these NaN entries in the original DataFrame
indices_to_update = unanswered_df.index[:batch_size]

# Prepare prompt content for the first 10 entries with NaN answer_string
user_messages = [format_user_message(text) for text in unanswered_df['text_w_eos'].iloc[:batch_size]]

# Save the indices list if needed for later use
indices_to_update_list = list(indices_to_update)

messages = [ system_message+" \n "+user_message for user_message in user_messages]
 

responses = []
for i in range(len(messages)):
    sequences = pipeline(
        messages[i],
        max_new_tokens=512,
        do_sample=False,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

    responses.append(seq['generated_text'])

# Display and process responses in a loop
for i, response in enumerate(responses):
    print(response)
    

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Result: Does the paragraph mention any of the following topics:
  (1) relation: Family and social relationships.
  (2) protein: High protein diet, carbohydrate-reduced(low-carb) high-protein diet.
  (3) ed: Eating disorders(ED) diagnosis or recovery, ED includes anorexia nervosa, anorexic, bulimia, bulimic, binge eating disorders, arfid, osfed, pica.
  (4) exercise: Physical exercise.
  (5) meal: Routine of meals.
  (6) crave: Craving for high calorie food or carbs.
  (7) restrict: Restrict nutrition or calorie intake.
  (8) binge: Binge eating.
  (9) loss: Body weight loss.
  (10) gain: Body weight gain.
  (11) calorie: Count calorie.
  (12) thinspo: Drive for thinness, want to be thinner or skinny.
  (13) leanbody: Drive for lean body mass, low body fat, more muscular or muscles. Must related to body shape..
  (14) bodyhate: Body dissatisfaction, feel bad about body image and appearrance. Must related to body image..
  (15) feargain: Fear of body weight gain. Must related to body wei

In [8]:
sequences

[{'generated_text': "\nYou are an AI assistant designed to answer questions.\nPlease restrict your answer to the exact question and use the exact answer format asked.\nAnswer should not have implied information. If the answer is yes, always provide related phrases. \n \n Does the paragraph mention any of the following topics:\n  (1) relation: Family and social relationships.\n  (2) protein: High protein diet, carbohydrate-reduced(low-carb) high-protein diet.\n  (3) ed: Eating disorders(ED) diagnosis or recovery, ED includes anorexia nervosa, anorexic, bulimia, bulimic, binge eating disorders, arfid, osfed, pica.\n  (4) exercise: Physical exercise.\n  (5) meal: Routine of meals.\n  (6) crave: Craving for high calorie food or carbs.\n  (7) restrict: Restrict nutrition or calorie intake.\n  (8) binge: Binge eating.\n  (9) loss: Body weight loss.\n  (10) gain: Body weight gain.\n  (11) calorie: Count calorie.\n  (12) thinspo: Drive for thinness, want to be thinner or skinny.\n  (13) leanbo